## Implementing a LSTM based Word generator from Simpsons Movie Scripts.
The strategy is suggested by Kaushal Shetty here https://github.com/keras-team/keras/issues/2009

In [1]:
import numpy as np
import pandas as pd

import collections

import re, nltk

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation

from keras.layers.embeddings import Embedding

# Visualization
import seaborn as sns

# this allows plots to appear directly in the notebook
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
INPUT_FILE = "data/moes_tavern_lines.txt"

In [3]:
content = open(INPUT_FILE).read()

In [4]:
content[:500]

"[YEAR DATE 1989] Â© Twentieth Century Fox Film Corporation. All rights reserved.\n\nMoe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szys"

In [5]:
from keras.preprocessing.text import text_to_word_sequence
word_sequence = text_to_word_sequence(content)

We take a window size of SEQLEN and create a new training data set each of SEQLEN words and (SEQLEN+1) th word becomes our label.

In [6]:
SEQLEN = 10

In [7]:
input_sentences = []
target_words = []
for i in range(0, len(word_sequence) - SEQLEN):
    input_sentences.append(" ".join(word_sequence[i:i + SEQLEN]))
    target_words.append(word_sequence[i + SEQLEN])

In [8]:
input_sentences[:10]

['year date 1989 â© twentieth century fox film corporation all',
 'date 1989 â© twentieth century fox film corporation all rights',
 '1989 â© twentieth century fox film corporation all rights reserved',
 'â© twentieth century fox film corporation all rights reserved moe',
 'twentieth century fox film corporation all rights reserved moe szyslak',
 'century fox film corporation all rights reserved moe szyslak into',
 'fox film corporation all rights reserved moe szyslak into phone',
 "film corporation all rights reserved moe szyslak into phone moe's",
 "corporation all rights reserved moe szyslak into phone moe's tavern",
 "all rights reserved moe szyslak into phone moe's tavern where"]

In [9]:
target_words[:10]

['rights',
 'reserved',
 'moe',
 'szyslak',
 'into',
 'phone',
 "moe's",
 'tavern',
 'where',
 'the']

Use keras tokenizer to tokenize the texts and do a texts_to_sequences on them

In [10]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [11]:
tokenizer.fit_on_texts(input_sentences)

In [12]:
input_sequences = tokenizer.texts_to_sequences(input_sentences)
input_sequences[:10]

[[467, 503, 6453, 6452, 2927, 1942, 1217, 1490, 1941, 30],
 [503, 6453, 6452, 2927, 1942, 1217, 1490, 1941, 30, 2928],
 [6453, 6452, 2927, 1942, 1217, 1490, 1941, 30, 2928, 1943],
 [6452, 2927, 1942, 1217, 1490, 1941, 30, 2928, 1943, 1],
 [2927, 1942, 1217, 1490, 1941, 30, 2928, 1943, 1, 7],
 [1942, 1217, 1490, 1941, 30, 2928, 1943, 1, 7, 119],
 [1217, 1490, 1941, 30, 2928, 1943, 1, 7, 119, 172],
 [1490, 1941, 30, 2928, 1943, 1, 7, 119, 172, 162],
 [1941, 30, 2928, 1943, 1, 7, 119, 172, 162, 343],
 [30, 2928, 1943, 1, 7, 119, 172, 162, 343, 134]]

In [13]:
np.array(input_sequences).shape

(53158, 10)

Map target_words to tokenizer.word_index and convert it into a categorical variable.

In [14]:
target_sequences = list(map(lambda w: tokenizer.word_index[w], target_words))
target_sequences[:10]

[2928, 1943, 1, 7, 119, 172, 162, 343, 134, 2]

In [15]:
np.array(target_sequences).shape

(53158,)

In [16]:
# Investigate first two labels
list(map(lambda w: tokenizer.word_index[w], ['rights', 'reserved']))

[2928, 1943]

In [17]:
word2index = tokenizer.word_index
index2word = {v:k for k, v in word2index.items()}

In [18]:
VOCAB_SIZE = len(word2index) + 1

In [19]:
VOCAB_SIZE

6454

In [20]:
target_one_hot = np.zeros((len(target_sequences), VOCAB_SIZE))
for i, wid in enumerate(target_sequences):
    target_one_hot[i, wid] = 1

In [21]:
target_one_hot

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
target_one_hot.shape

(53158, 6454)

### Using Embedding Layer

Since the vocabulary is very large the numerical sequences turn into sparse arrays and it's more efficient to cast everything to a lower dimension with the `Embedding` layer.

In [23]:
EMBED_SIZE  = 300
HIDDEN_SIZE = 256

In [24]:
# Build model
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, 
                    output_dim=EMBED_SIZE, 
                    input_length=SEQLEN, 
                    weights=None,
                    trainable=True))
model.add(LSTM(units=HIDDEN_SIZE, return_sequences=True, unroll=True))
model.add(Dropout(0.2))
model.add(LSTM(units=HIDDEN_SIZE, return_sequences=True, unroll=True))
model.add(Dropout(0.2))
model.add(LSTM(units=HIDDEN_SIZE, return_sequences=False, unroll=True))
model.add(Dropout(0.2))
model.add(Dense(VOCAB_SIZE))
model.add(Activation("softmax"))

In [25]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [26]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 300)           1936200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 256)           570368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [27]:
BATCH_SIZE = 128
NUM_ITERATIONS = 50
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 50

Since we don't have any labeled data here, we train the model for an epoch (`NUM_EPOCHS_PER_ITERATION=1`) then test it. We continue training like this for 25 (`NUM_ITERATIONS=25`) iterations, stopping once we see intelligible output. So effectively, we are training for `NUM_ITERATIONS` epochs and testing the model after each epoch.

Our test consists of generating a word from the model given a random input, then dropping the first word from the input and appending the predicted word from our previous run, and generating another word from the model. We continue this 100 times (`NUM_PREDS_PER_EPOCH=100`) and generate and print the resulting string. The string gives us an indication of the quality of the model:

In [28]:
X = np.array(input_sequences)
y = target_one_hot

In [29]:
X.shape, y.shape

((53158, 10), (53158, 6454))

In [30]:
test_sequence = input_sequences[0]
test_sequence

[467, 503, 6453, 6452, 2927, 1942, 1217, 1490, 1941, 30]

In [31]:
test_sentence = input_sentences[0] #" ".join(list(map(lambda wid: index2word[wid], test_sequence)))
test_sentence

'year date 1989 â© twentieth century fox film corporation all'

In [32]:
test_sequence = np.array(test_sequence).reshape(1, 10)
test_sequence

array([[ 467,  503, 6453, 6452, 2927, 1942, 1217, 1490, 1941,   30]])

In [33]:
test_sequence.shape

(1, 10)

In [34]:
input_sentences[10699]

"work without the fork in the eye moe szyslak there's"

In [35]:
# We train the model in batches and test output generated at each step
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    
    model.fit(X, y, epochs=NUM_EPOCHS_PER_ITERATION) # train for 1 epoch
    
    # testing model
    # randomly choose a row from input_sentences, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_sequences))
    print(test_idx)
    test_sequence = input_sequences[test_idx]
    #test_sentence = " ".join(list(map(lambda wid: index2word[wid], test_sequence)))
    test_sentence = input_sentences[test_idx]
    print("Generating from seed: %s" % test_sentence)
    print(test_sentence, end=" ")
    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.array(test_sequence).reshape(1, SEQLEN)
        #X_test = np.zeros((1, SEQLEN, nb_words))
        #for i, word in enumerate(test_words):
        #    X_test[0, i, word2index[word]] = 1
        pred = model.predict(X_test, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        # move forward with test_chars + ypred
        #test_words = test_words[1:].append(ypred)
        del test_sequence[0]
        test_sequence.append(np.argmax(pred))
print()

Iteration #: 0
Epoch 1/1
53158/53158 [==============================] - 109s 2ms/step - loss: 6.8814
818
Generating from seed: a purveyor of mind numbing intoxicants like myself homer simpson
a purveyor of mind numbing intoxicants like myself homer simpson i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i ==================================================
Iteration #: 1
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 7.2294
51149
Generating from seed: crummy little kids that nobody can control homer simpson you
crummy little kids that nobody can control homer simpson you i moe moe moe i i i i i moe i moe moe moe i i i i i moe i moe moe moe i i i i i moe i moe moe moe i i i i i moe i moe moe moe i i i i i moe ==================================================
Iteration #: 2
Epoch 1/1
53158/53158 [==============================] - 105s 2ms/step - loss: 6.5965
38541
Generating from seed: head in ominous the r

brag about it but i did publish five modestly successful i know i have a big time and i have a big time is i have a time and i have a big time is what i know i never never have a big time moe szyslak yeah i am get a time moe szyslak oh i can't get a ==================================================
Iteration #: 17
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 6.9204
37647
Generating from seed: it's official smithers is now worse than mr burns lenny
it's official smithers is now worse than mr burns lenny leonard well i know you i want to go on your money moe szyslak yeah that's your time barney gumble oh i can have a money moe szyslak yeah yeah i'm gonna have a new new moe szyslak oh well that's your time homer simpson i can have to get ==================================================
Iteration #: 18
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 6.8879
34853
Generating from seed: kent brockman with a channel 6 exclusiv

i pay with a drawing moe szyslak yeah nice try he moe szyslak and no more is not gonna do you have is gonna have to get your beer moe szyslak oh yeah i'm not right for a good thing homer simpson all not not all a good beer moe szyslak all right all right i don't want to get ==================================================
Iteration #: 33
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 6.1712
15435
Generating from seed: paid your tab or at least cleaned up that mess
paid your tab or at least cleaned up that mess you go for me homer lenny leonard i'm gonna come in your beer homer simpson no so no that's here homer simpson well if i'm gonna go to my wife carl carlson you know that is beer carl carlson like you say i need to get for my beer and ==================================================
Iteration #: 34
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 6.1311
45116
Generating from seed: for loudly helllp normal 'cause thi

best friend homer simpson rainier wolfcastle carl carlson teenage girl tough funny lenny leonard he are that carl carlson but they just think marge simpson fine i'm just here a best date and you moe szyslak okay so one hundred one little little little aw homer simpson marge don't you think i think like that man like you back homer ==================================================
Iteration #: 49
Epoch 1/1
53158/53158 [==============================] - 103s 2ms/step - loss: 5.5127
22710
Generating from seed: my head he's like a spy in the house of
my head he's like a spy in the house of you man is your best thing i'm not right barney homer simpson no man okay how like you got a moe szyslak ah that's this beer homer simpson so i'm not here we want to moe szyslak i'm not a moe szyslak oh yeah no good man is so good 


### Using Embedding Layer initialised with pre-trained GloVe embeddings

In [36]:
GLOVE_MODEL = "data/glove.6B.300d.txt"

In [37]:
# load GloVe vectors
word2emb = {}
with open(GLOVE_MODEL, "rb") as fglove:
    for line in fglove:
        cols = line.strip().split()
        word = cols[0]
        embedding = np.array(cols[1:], dtype="float32")
        word2emb[word] = embedding

In [38]:
glove_embedding_weights = np.zeros((VOCAB_SIZE, EMBED_SIZE))
for word, index in word2index.items():
    try:
        glove_embedding_weights[index, :] = word2emb[word]
    except KeyError:
        # words not found in glove embedding dictionary word2emb will be all-zeros.
        pass

In [39]:
VOCAB_SIZE = len(word2index) + 1
EMBED_SIZE  = 300
HIDDEN_SIZE = 128

In [41]:
# Build model
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, 
                    output_dim=EMBED_SIZE, 
                    input_length=SEQLEN, 
                    weights=[glove_embedding_weights], 
                    trainable=True))
model.add(LSTM(units=HIDDEN_SIZE, return_sequences=True, unroll=True))
model.add(Dropout(0.2))
model.add(LSTM(units=HIDDEN_SIZE, return_sequences=False, unroll=True))
model.add(Dropout(0.2))
model.add(Dense(VOCAB_SIZE))
model.add(Activation("softmax"))

In [42]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [43]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 300)           1936200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 128)           219648    
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6454)              832566    
_________________________________________________________________
activation_2 (Activation)    (None, 6454)              0         
Total para

In [44]:
BATCH_SIZE = 128
NUM_ITERATIONS = 50
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 50

In [45]:
# We train the model in batches and test output generated at each step
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    
    model.fit(X, y, epochs=NUM_EPOCHS_PER_ITERATION) # train for 1 epoch
    
    # testing model
    # randomly choose a row from input_sentences, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_sequences))
    print(test_idx)
    test_sequence = input_sequences[test_idx]
    #test_sentence = " ".join(list(map(lambda wid: index2word[wid], test_sequence)))
    test_sentence = input_sentences[test_idx]
    print("Generating from seed: %s" % test_sentence)
    print(test_sentence, end=" ")
    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.array(test_sequence).reshape(1, SEQLEN)
        #X_test = np.zeros((1, SEQLEN, nb_words))
        #for i, word in enumerate(test_words):
        #    X_test[0, i, word2index[word]] = 1
        pred = model.predict(X_test, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred, end=" ")
        # move forward with test_chars + ypred
        #test_words = test_words[1:].append(ypred)
        del test_sequence[0]
        test_sequence.append(np.argmax(pred))
print()

Iteration #: 0
Epoch 1/1
53158/53158 [==============================] - 62s 1ms/step - loss: 6.5442
30280
Generating from seed: it was the first day of summer moe szyslak i
it was the first day of summer moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe szyslak i i moe ==================================================
Iteration #: 1
Epoch 1/1
53158/53158 [==============================] - 60s 1ms/step - loss: 6.2621
20338
Generating from seed: the ol' eye gouger in the pickle brine that'll keep
the ol' eye gouger in the pickle brine that'll keep moe szyslak hey i i you you you to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to ==================================================
Iteration #: 2
Epoch 1/1
53158/53158 [==============================] - 59s 1ms/step - loss:

simpson um uh attempting to change subject so uh joey man can do they get a drink moe szyslak oh yeah uh uh uh don't know it was good for a way to get one thing moe szyslak no uh look like a little guy but i can't see it was good to the big drink but i'm gonna get ==================================================
Iteration #: 17
Epoch 1/1
53158/53158 [==============================] - 60s 1ms/step - loss: 6.5138
7032
Generating from seed: but you know we had good writers william faulkner could
but you know we had good writers william faulkner could make it as a little guy at the bar marge simpson okay i'm not not a little guy moe szyslak oh oh yeah no homer simpson no no no no no no no no no uh okay hey ah okay if you see that time i'm gonna get it for ==================================================
Iteration #: 18
Epoch 1/1
53158/53158 [==============================] - 61s 1ms/step - loss: 6.5301
31425
Generating from seed: i just had this awful feeling that uh homer's in
i just 

clear in boston lenny leonard to homer i brought a bar on a bar bart simpson okay well i was one like that bar marge simpson good one hey moe i can't get out of your bar to get your best friend moe szyslak oh my god no was not not too good more carl carlson well uh no can't ==================================================
Iteration #: 33
Epoch 1/1
53158/53158 [==============================] - 64s 1ms/step - loss: 6.2694
20617
Generating from seed: trust you how'd you like to be a snitch the
trust you how'd you like to be a snitch the gotta do that was at the bar marge simpson okay okay i think i was the bar now to go with you don't get out of the man now what do you can be one of those bar who can't buy that in the bar homer simpson okay now moe ==================================================
Iteration #: 34
Epoch 1/1
53158/53158 [==============================] - 64s 1ms/step - loss: 6.2650
4643
Generating from seed: problemo points hey moe look over there moe szyslak what
probl

it's such a lovely blend barney gumble sings in harmony it's is no i can have no little uh oh marge simpson uh uh no uh just come on here lenny leonard i'm just just like a bar to get your wife is so one of your bar man who could have to do marge simpson oh my god now ==================================================
Iteration #: 49
Epoch 1/1
53158/53158 [==============================] - 64s 1ms/step - loss: 5.9793
52943
Generating from seed: szyslak beat just a minute let me check to the
szyslak beat just a minute let me check to the little bar oh well then what are you one homer simpson okay i'm not back to go carl carlson hey now all is a bar more time homer simpson i'm not here moe szyslak who can never get your beer here moe szyslak well i'm not so so no man 
